In [3]:
import numpy as np
import pandas as pd
import math
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.base import TransformerMixin
import itertools

# slotprice, non-linear, gradiant boosting tree

In [34]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
validation = pd.read_csv('validation.csv')

In [35]:
train.click.value_counts()

0    2429188
1       1793
Name: click, dtype: int64

In [36]:
total = pd.concat([train, validation, test])

/Users/sue/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [37]:
total.head()

,IP,adexchange,advertiser,bidid,bidprice,city,click,creative,domain,hour,...,slotid,slotprice,slotvisibility,slotwidth,url,urlid,useragent,userid,usertag,weekday
0,125.37.175.*,2.0,3427,b7bea80521fdecd95d2d761a38c91c3f09618066,238.0,2,0.0,a4f763f78ef3eedfe614263b94a8924e,trqRTvKaXTKfgg24JKTI,22,...,2015392487,5,2,200,6447a7dfa30fe6eb410c91860e7c9b45,NaN,windows_ie,2e880fb7d690cf7377b2e42e701728e3f3c0e4c1,NaN,5
1,171.36.92.*,1.0,2821,4f51205475678f5a124bc76b2c54163bf8eaa7eb,294.0,239,0.0,10722,20fc675468712705dbf5d3eda94126da,20,...,mm_10982364_973726_8930541,0,FourthView,300,3ddf173a94bd23c326683b6373c75dd4,NaN,windows_chrome,3a1fe01360ff8100e7d006b83b77a3e4c01d928c,NaN,1
2,59.46.106.*,2.0,3427,b604e3fd054a658ab7ced4285ebf2ef54d2bd890,238.0,41,0.0,798b2d49952d77f1eace9f23c210d0b5,trqRTJn7O95I1mKYUV,13,...,1720123646,5,2,250,625d1b5916ea925332c7b326c0574cfa,NaN,windows_ie,801d18a056b6fe6b06a794aef17fb0d6daff2414,"10052,10006,13866,10110",3
3,114.250.226.*,1.0,1458,0348beeae93e561584c3b50fc9e7746a33048ad7,300.0,1,0.0,cb7c76e7784031272e37af8e7e9b062c,5F97t5E0BTK7XhNrUMpENpn,23,...,mm_10027070_118039_10308280,0,2,160,dedc488b98ca20707bc9a723957e7d1f,NaN,windows_ie,0d6eaf2259699990e38a1fc5116f112070b9ecdc,"13866,10063,10111",6
4,183.63.192.*,2.0,2259,268149c1789bce2bc9798ffd97ec431219bafeb3,277.0,233,0.0,7330,13625cb070ffb306b425cd803c4b7ab4,6,...,1120200150,133,OtherView,728,4199d1227c511fc776b76594dabff9f8,NaN,windows_ie,a239d9bb642460d974ba67f85e63b8d3e214da0e,NaN,5


In [38]:
total["usertag"] = total["usertag"].fillna("") 

In [39]:
def UsertagCategories(df):
    
    # Drop nan
    df = df["usertag"].dropna().reset_index(drop = True)
    
    # Find unique usertags
    usertags_list = [df[i].split(",") for i in range(df.shape[0])]
    
    # itertools.chain.from_iterable joins a list of lists into a single list
    usertags = np.unique(list(itertools.chain.from_iterable(usertags_list)))
    
    # Remove the empty string ""
    usertags = [tag for tag in usertags if len(tag) > 0]
    
    return usertags

In [40]:
usertags_train = UsertagCategories(total)

In [41]:
for tag in usertags_train:
    col_name = "usertag_" + tag
    total[col_name] = total["usertag"].map(lambda x: 1 if tag in x.split(",") else 0)


In [42]:
total.columns

Index(['IP', 'adexchange', 'advertiser', 'bidid', 'bidprice', 'city', 'click',
       'creative', 'domain', 'hour', 'keypage', 'payprice', 'region',
       'slotformat', 'slotheight', 'slotid', 'slotprice', 'slotvisibility',
       'slotwidth', 'url', 'urlid', 'useragent', 'userid', 'usertag',
       'weekday', 'usertag_10006', 'usertag_10024', 'usertag_10031',
       'usertag_10048', 'usertag_10052', 'usertag_10057', 'usertag_10059',
       'usertag_10063', 'usertag_10067', 'usertag_10074', 'usertag_10075',
       'usertag_10076', 'usertag_10077', 'usertag_10079', 'usertag_10083',
       'usertag_10093', 'usertag_10102', 'usertag_10110', 'usertag_10111',
       'usertag_10114', 'usertag_10115', 'usertag_10116', 'usertag_10117',
       'usertag_10118', 'usertag_10120', 'usertag_10123', 'usertag_10125',
       'usertag_10126', 'usertag_10127', 'usertag_10129', 'usertag_10130',
       'usertag_10131', 'usertag_10133', 'usertag_10138', 'usertag_10140',
       'usertag_10142', 'usertag_101

In [43]:
# split useragent into system and browser
total["system"] = total["useragent"].map(lambda x: x.split("_")[0])
total["browser"] = total["useragent"].map(lambda x: x.split("_")[1])

In [44]:
#delete too specific variables and irrelevant variables 
remove =['bidid', 'userid', 'IP', 'domain', 'url', 'urlid', 'slotid', 'useragent','keypage','creative','usertag']
#remove =['bidid', 'userid', 'IP', 'domain', 'url', 'urlid', 'slotid', 'useragent','keypage','creative','usertag','city'] 
total = total.drop(remove, axis=1)

In [45]:
total.columns

Index(['adexchange', 'advertiser', 'bidprice', 'city', 'click', 'hour',
       'payprice', 'region', 'slotformat', 'slotheight', 'slotprice',
       'slotvisibility', 'slotwidth', 'weekday', 'usertag_10006',
       'usertag_10024', 'usertag_10031', 'usertag_10048', 'usertag_10052',
       'usertag_10057', 'usertag_10059', 'usertag_10063', 'usertag_10067',
       'usertag_10074', 'usertag_10075', 'usertag_10076', 'usertag_10077',
       'usertag_10079', 'usertag_10083', 'usertag_10093', 'usertag_10102',
       'usertag_10110', 'usertag_10111', 'usertag_10114', 'usertag_10115',
       'usertag_10116', 'usertag_10117', 'usertag_10118', 'usertag_10120',
       'usertag_10123', 'usertag_10125', 'usertag_10126', 'usertag_10127',
       'usertag_10129', 'usertag_10130', 'usertag_10131', 'usertag_10133',
       'usertag_10138', 'usertag_10140', 'usertag_10142', 'usertag_10145',
       'usertag_10146', 'usertag_10147', 'usertag_10148', 'usertag_10149',
       'usertag_10684', 'usertag_11092', '

In [46]:
# onehot encoding 
columns = ['weekday', 'hour', 'region', 'slotvisibility', 'slotformat', 'system','browser','advertiser','adexchange','city']
#columns = ['weekday', 'hour', 'region', 'slotvisibility', 'slotformat', 'system','browser', 'slotwidth','slotheight', 'slotprice','advertiser','adexchange']
#columns = ['weekday', 'hour', 'region', 'slotvisibility', 'slotformat', 'system','browser', 'slotwidth','slotheight', 'slotprice','advertiser','adexchange','city']
def onehot_encoding(dataset,columns):
    dataset = pd.get_dummies(dataset, columns=columns)
    return dataset


In [47]:
total = onehot_encoding(total,columns)

In [48]:
total.head()

,bidprice,click,payprice,slotheight,slotprice,slotwidth,usertag_10006,usertag_10024,usertag_10031,usertag_10048,...,city_386,city_387,city_388,city_393,city_394,city_395,city_396,city_397,city_398,city_399
0,238.0,0.0,5.0,200,5,200,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,294.0,0.0,23.0,250,0,300,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,238.0,0.0,24.0,250,5,250,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,300.0,0.0,25.0,600,0,160,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,277.0,0.0,133.0,90,133,728,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
# transfer nonnumeric variables 
total.select_dtypes(include=['object']).columns

def label_encoder(dataset, columns):
    
    transform_dict = {}
    for col in columns_for_enconding:
        cats = pd.Categorical(dataset[col]).categories
        d = {}
        for i, cat in enumerate(cats):
            d[cat] = i
        transform_dict[col] = d

    # Create an inverse dictionary
    inverse_transform_dict = {}
    for col, d in transform_dict.items():
        inverse_transform_dict[col] = {v: k for k, v in d.items()}

    # Replace the odd values in initial dataset
    dataset[columns_for_enconding] = dataset[columns_for_enconding].replace(transform_dict)

    return dataset

In [78]:
# numerical varibales 
columns = ['slotwidth', 'slotheight', 'slotprice']
def min_max_scaling(dataset,columns):
    scaler = MinMaxScaler()
    dataset[columns] = pd.DataFrame(scaler.fit_transform(dataset[columns]))
    return dataset

In [49]:
# split total dataset to train and validation 
train_final = total[:len(train)]
validation_final = total[len(train): len(train)+len(validation)]
test_final = total[-len(test):]

In [50]:
len(train_final)

2430981

In [51]:
len(validation_final)

303925

In [52]:
len(test_final)

303375

In [53]:
# the train dataset is highly imbalance , implement the downsampling
def downsampling(dataset):
    class_ratio = 0.05
    seed=500
    train_large = dataset.loc[dataset["click"] == 0]
    train_small = dataset.loc[dataset["click"] == 1]
    large_len= math.floor(len(dataset)*(len(train_small)/len(dataset))/class_ratio-len(train_small))
    l_downsampled = resample(train_large,
                                         replace=False,
                                         n_samples=large_len,
                                         random_state=seed)
    dataset = pd.concat([train_small, l_downsampled])
    return dataset

In [54]:
train_final = downsampling(train_final)

In [55]:
train_final.head()

,bidprice,click,payprice,slotheight,slotprice,slotwidth,usertag_10006,usertag_10024,usertag_10031,usertag_10048,...,city_386,city_387,city_388,city_393,city_394,city_395,city_396,city_397,city_398,city_399
1104,300.0,1.0,80.0,90,80,1000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4949,300.0,1.0,65.0,280,0,336,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8878,300.0,1.0,101.0,250,0,300,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9790,277.0,1.0,160.0,50,160,320,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000,300.0,1.0,108.0,250,0,300,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
train_final.shape

(35860, 553)

In [57]:
train_final.click.value_counts()

0.0    34067
1.0     1793
Name: click, dtype: int64

In [58]:
train_final.to_csv("new_train.csv")
validation_final.to_csv("new_validation.csv")
test_final.to_csv('new_test.csv')

In [4]:
click = pd.read_csv('XG_pClick.csv')

In [8]:
click.head()

,Unnamed: 0,bidid,pClick
0,0,366c563de7d90feb9d4dab53e795a93fb3157387,0.0
1,1,29167d4caa719788b5a342dbaa25151d53121f80,0.0
2,2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,0.0
3,3,844c2da00d45315f20b748ec131c26ee99a7cbc7,0.0
4,4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,0.0


In [7]:
click[].value_counts()

AttributeError: 'DataFrame' object has no attribute 'value_counts'